# 📊 QEPC Player Props – Quick Test & Backtest

This notebook runs the **player props engine** and a simple **backtest** using your `props` package:

- Auto-detects the `qepc_project` root (looks for `props/` and `data/`).
- Imports `PlayerPropsEngine` and `PropsBacktestEngine` from `props/`.
- Runs a **quick single-player prediction** as a smoke test.
- Runs a **backtest** over a configurable date range and prints a summary.

> You can safely edit the player name, opponent, and backtest dates in the cells below.


In [ ]:
# 🧱 Cell 1 – Find project root and add to sys.path

import sys
from pathlib import Path

here = Path.cwd().resolve()
project_root = None

for p in [here] + list(here.parents):
    if (p / "props").exists() and (p / "data").exists():
        project_root = p
        break

if project_root is None:
    raise RuntimeError(
        "Could not find project root with props/ and data/ folders.\n"
        f"Started from: {here}"
    )

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Project root set to: {project_root}")
print(f"   sys.path[0]: {sys.path[0]}")

In [2]:
# 🧩 Cell 2 – Imports and configuration

from datetime import date, timedelta
from pathlib import Path

import pandas as pd
import numpy as np

try:
    from props.player_props_engine import PlayerPropsEngine, PropPrediction
    from props.props_backtest_engine import PropsBacktestEngine, BacktestSummary
    print("✅ Imported PlayerPropsEngine and PropsBacktestEngine from props/")
except ImportError as e:
    print(f"❌ ImportError: {e}")
    print("Make sure this notebook is inside your qepc_project tree,")
    print("and that the previous cell correctly found the project root.")
    raise

# Let the engines auto-detect PlayerStatistics.csv location
DATA_PATH = None  # keep as None unless you want to force a specific CSV path

# Backtest configuration (you can change these)
BACKTEST_START = "2024-11-01"
BACKTEST_END   = "2024-11-15"
PROPS_TO_TEST  = ["PTS", "REB", "AST", "3PM", "PRA"]
MIN_MINUTES    = 15.0

print("\n📊 Backtest configuration:")
print(f"   Data override: {DATA_PATH!r} (None = auto-detect)")
print(f"   Date range:   {BACKTEST_START} → {BACKTEST_END}")
print(f"   Props:        {PROPS_TO_TEST}")
print(f"   Min minutes:  {MIN_MINUTES}")

❌ ImportError: No module named 'player_props_engine'
Make sure this notebook is inside your qepc_project tree,
and that the previous cell correctly found the project root.


ModuleNotFoundError: No module named 'player_props_engine'

In [ ]:
# 🧪 Cell 3 – Quick single-player test

# Adjust these to a player + opponent that you know exist in PlayerStatistics.csv
TEST_PLAYER   = "LeBron James"
TEST_OPPONENT = "Golden State Warriors"  # should match opponentteamName in your CSV

print("\n" + "="*60)
print("🧪 QUICK TEST – Single Player")
print("="*60)

engine = PlayerPropsEngine(DATA_PATH)
engine.load_data()

print(f"\n📈 Testing {TEST_PLAYER} vs {TEST_OPPONENT}")
for prop in ["PTS", "REB", "AST", "PRA"]:
    pred = engine.predict(
        player_name=TEST_PLAYER,
        prop_type=prop,
        opponent=TEST_OPPONENT,
        is_home=True,
    )
    if pred is None:
        print(f"\n{prop}: Player or data not found.")
        continue

    print(f"\n{prop}:")
    print(f"  Projection: {pred.projection:.2f}")
    print(f"  Floor/Ceiling: {pred.floor:.1f} – {pred.ceiling:.1f}")
    print(f"  Confidence: {pred.confidence}")
    if prop == "PTS":
        line = 25.5
        print(f"  P({TEST_PLAYER} > {line}) = {pred.over_prob(line):.1%}")

In [ ]:
# 🚀 Cell 4 – Full backtest and summary

print("\n" + "="*60)
print("🚀 RUNNING FULL BACKTEST")
print("="*60)

backtest = PropsBacktestEngine(DATA_PATH)

results = backtest.run_backtest(
    start_date=BACKTEST_START,
    end_date=BACKTEST_END,
    props=PROPS_TO_TEST,
    min_minutes=MIN_MINUTES,
    verbose=True,
)

if not results:
    print("\n❌ No backtest results returned. Check date range and data availability.")
else:
    print(f"\n✅ Backtest complete: {len(results)} predictions generated")

    summary = backtest.get_summary()

    print("\n" + "="*60)
    print("📊 BACKTEST SUMMARY")
    print("="*60)

    print(f"""
┌─────────────────────────────────────────────┐
│           OVERALL PERFORMANCE               │
├─────────────────────────────────────────────┤
│  Total Predictions:    {summary.total_predictions:>6}              │
│  Mean Absolute Error:  {summary.mean_absolute_error:>6.2f} pts     │
│  Median Abs Error:     {summary.median_absolute_error:>6.2f} pts   │
│  Mean % Error:         {summary.mean_pct_error*100:>6.1f}%         │
├─────────────────────────────────────────────┤
│           DIRECTIONAL ACCURACY              │
├─────────────────────────────────────────────┤
│  Overall Accuracy:     {summary.overall_accuracy*100:>6.1f}%       │
│  Over Accuracy:        {summary.over_accuracy*100:>6.1f}%          │
│  Under Accuracy:       {summary.under_accuracy*100:>6.1f}%         │
├─────────────────────────────────────────────┤
│           CALIBRATION                       │
├─────────────────────────────────────────────┤
│  Brier Score:          {summary.brier_score:>6.4f}                 │
│  (Lower is better, 0.25 = random)           │
├─────────────────────────────────────────────┤
│           BY CONFIDENCE                     │
├─────────────────────────────────────────────┤
│  HIGH Confidence:      {summary.high_conf_accuracy*100:>6.1f}%     │
│  MEDIUM Confidence:    {summary.medium_conf_accuracy*100:>6.1f}%   │
│  LOW Confidence:       {summary.low_conf_accuracy*100:>6.1f}%      │
├─────────────────────────────────────────────┤
│           SIMULATED BETTING                 │
├─────────────────────────────────────────────┤
│  Bets Placed:          {summary.simulated_bets:>6}                 │
│  Bets Won:             {summary.simulated_wins:>6}                 │
│  ROI:                  {summary.simulated_roi*100:>6.1f}%          │
└─────────────────────────────────────────────┘
    """)